# Pt. 1 Initial intro to the datasets

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import dash
import os


In [14]:
first_impression = {}
filenames = os.listdir('../data')

In [15]:
filenames

['sat_2019.csv',
 'act_2018.csv',
 'act_2019_ca.csv',
 'sat_act_by_college.csv',
 'act_2019.csv',
 'sat_2019_by_intended_college_major.csv',
 'sat_2017.csv',
 'sat_2019_ca.csv',
 'act_2017.csv',
 'sat_2018.csv']

In [26]:
first_impression = {file : pd.read_csv(f"../data/{file}") for file in filenames}

In [33]:
##ok lets narrow i down a bit

for year in range(2017, 2020):
    key = f'sat_{year}.csv'
    print(first_impression[key].columns)

Index(['State', 'Participation', 'Evidence-Based Reading and Writing', 'Math',
       'Total'],
      dtype='object')
Index(['State', 'Participation', 'Evidence-Based Reading and Writing', 'Math',
       'Total'],
      dtype='object')
Index(['State', 'Participation Rate', 'EBRW', 'Math', 'Total'], dtype='object')


In [40]:
#so SAT dfs all have the same catagories, except that one that needs to be renamed, im not really manipulating the data yet, but lets fix that
first_impression[key].rename({'EBRW': 'Evidence-Based Reading and Writing'}, axis = 1, inplace = True)



In [42]:
#repeat for ACT 
for year in range(2017, 2020):
    key = f'act_{year}.csv'
    print(first_impression[key].columns)

Index(['State', 'Participation', 'English', 'Math', 'Reading', 'Science',
       'Composite'],
      dtype='object')
Index(['State', 'Participation', 'Composite'], dtype='object')
Index(['State', 'Participation', 'Composite'], dtype='object')


In [46]:
#im hoping we have all 50 states for both tests. quickest way to check:
tests = ['act','sat']
for test in tests:
    print(test)
    for year in range(2017, 2020):
        key = f"{test}_{year}.csv"
        print(first_impression[key].shape)

act
(52, 7)
(52, 3)
(52, 3)
sat
(51, 5)
(51, 5)
(53, 5)


In [44]:
#and what else do we have
first_impression['sat_act_by_college.csv'].head

<bound method NDFrame.head of                                     School Test Optional?  \
0                      Stanford University            Yes   
1                          Harvard College            Yes   
2                     Princeton University            Yes   
3                      Columbia University            Yes   
4                          Yale University            Yes   
..                                     ...            ...   
411  University of Texas Rio Grande Valley             No   
412             University of South Dakota             No   
413              University of Mississippi             No   
414                  University of Wyoming             No   
415            University of Texas El Paso             No   

    Applies to Class Year(s)  \
0                       2021   
1                       2021   
2                       2021   
3                       2021   
4                       2021   
..                       ...   
411          

## PLease see my starter-code, the cleaning_DFs, act_cleaning and sat_cleaning notebooks for info about getting the data prepaared. This noebook will focus on the Explaoratory Data Analysis)

In [26]:
sat_2017 = pd.read_pickle("../data/2017_sat.pkl")


In [27]:
s = sat_2017.sort_values(by = 'participation', ascending = False).head(10)

In [28]:
s

,state,participation,evidence-based_reading_and_writing,math,composite,year,rounded,percentile
8,District of Columbia,1.00,482,468,950,2017,950.0,0
22,Michigan,1.00,509,495,1005,2017,1000.0,0
6,Connecticut,1.00,530,512,1041,2017,1040.0,0
7,Delaware,1.00,503,492,996,2017,1000.0,0
29,New Hampshire,0.96,532,520,1052,2017,1050.0,0
19,Maine,0.95,513,499,1012,2017,1010.0,0
12,Idaho,0.93,513,493,1005,2017,1000.0,0
9,Florida,0.83,520,497,1017,2017,1020.0,0
21,Massachusetts,0.76,555,551,1107,2017,1110.0,0
39,Rhode Island,0.71,539,524,1062,2017,1060.0,0


In [30]:
s['state'].values

array(['District of Columbia', 'Michigan', 'Connecticut', 'Delaware',
       'New Hampshire', 'Maine', 'Idaho', 'Florida', 'Massachusetts',
       'Rhode Island'], dtype=object)

Looks like participation rate is negatively correlated with composite score. That's not too surprising, since lower participation draws from a college-bound population.

I'll graph them and see for sure in my visualizations notebook

In [32]:
s['participation'].values

array([1.  , 1.  , 1.  , 1.  , 0.96, 0.95, 0.93, 0.83, 0.76, 0.71])

The graphs show a clear negative correlation across tests and years. I conclude that the average SAT score of a state is inversely proporitonal to it's participation rate. Visually, I didn't find any especially interesting outliers. Lets build an index:
## Definition: Participation-score-index = composite score/ participation rate

In [13]:
act2016 = pd.read_pickle('../data/2016_act_with_percentiles.pkl')
act2016

In [16]:
act2016['participation_score_index'] = act2016['composite'].apply(lambda x : float(x))/act2016['participation']


In [5]:
act2016

NameError: name 'act2016' is not defined

In [58]:
filenames = [file for file in os.listdir('../data') if file.find('_with_p')!=-1]

In [59]:
def find_mean(values):
    total = 0
    n = len(values)
    for value in values:
        total += value
    return total/n

find_mean([3,3,3,3,3,3,3,5,5,5,5,5,5,5])

4.0

In [60]:
def standard_deviation(values):
    n = len(values)
    mu = find_mean(values)
    mse = 0
    for value in values:
        mse += (value - mu)**2
    return (mse/n)**.5

In [44]:
def index_info(file):
    df = pd.read_pickle(f'../data/{file}')
    df['participation_score_index'] = df['composite'].apply(lambda x : float(x))/df['participation']
    psi_mean = find_mean(df['participation_score_index'])
    psi_std = standard_deviation(df['participation_score_index'])
    tops = df.sort_values(by=['participation_score_index'], ascending = False).head()
    names = tops['state'].values
    psis = tops['participation_score_index'].values
    top_lib = {names[i] : psis[i] for i in range(5)}
    index_lib = {'test': file, 'mean_psi' : psi_mean, 'psi_standard': psi_std, 'top_five_states_by_psi': top_lib}
    return index_lib

In [45]:
index_info('2016_act_with_percentiles.pkl')

{'test': '2016_act_with_percentiles.pkl',
 'mean_psi': 53.004332626198874,
 'psi_standard': 49.41060103076636,
 'top_five_states_by_psi': {'Maine': 303.75,
  'New Hampshire': 141.66666666666669,
  'Delaware': 133.8888888888889,
  'Rhode Island': 114.28571428571429,
  'Pennsylvania': 103.04347826086956}}

In [48]:
indices  = [index_info(file) for file in filenames]

In [49]:
indices

[{'test': '2017_act_with_percentiles.pkl',
  'mean_psi': 64.13894059433147,
  'psi_standard': 68.00828357055413,
  'top_five_states_by_psi': {'Maine': 342.85714285714283,
   'Rhode Island': 161.33333333333334,
   'New Hampshire': 156.875,
   'Delaware': 140.0}},
 {'test': '2018_act_with_percentiles.pkl',
  'mean_psi': 67.50503201183123,
  'psi_standard': 68.73165781044507,
  'top_five_states_by_psi': {'Maine': 405.0,
   'Rhode Island': 205.83333333333334,
   'Delaware': 185.3846153846154,
   'New Hampshire': 178.57142857142856,
   'Pennsylvania': 138.8235294117647}},
 {'test': '2019_sat_with_percentile',
  'mean_psi': 12816.416479309099,
  'psi_standard': 16403.816494351147,
  'top_five_states_by_psi': {'North Dakota': 64150.0,
   'Wisconsin': 43133.333333333336,
   'Iowa': 42166.66666666667,
   'Wyoming': 41900.0,
   'Nebraska': 41733.333333333336}},
 {'test': '2018_sat_with_percentile',
  'mean_psi': 12278.27120972373,
  'psi_standard': 16285.779394712801,
  'top_five_states_by_psi':

This is great to compare within tests across years, but I want to compare across tests to so I need to normalize

(Thanks to Brad Soloman at https://stackoverflow.com/questions/46915495/normalization-vs-numpy-way-to-normalize for reminding me how to do that.)

In [61]:
def index_info_with_norm(file):
    df = pd.read_pickle(f'../data/{file}')
    df['participation_score_index'] = df['composite'].apply(lambda x : float(x))/df['participation']
    psi_mean = find_mean(df['participation_score_index'])
    psi_std = standard_deviation(df['participation_score_index'])
    df['norms'] = (df['participation_score_index'] - psi_mean)/psi_std
    tops = df.sort_values(by=['participation_score_index'], ascending = False).head()
    names = tops['state'].values
    psis = tops['participation_score_index'].values
    normal = tops['norms'].values
    top_lib = {names[i] : f'test_psi: {psis[i]}, normalized_psi: {normal[i]}' for i in range(5)}
    index_lib = {'test': file, 'mean_psi' : psi_mean, 'psi_standard': psi_std, 'top_five_states_by_psi': top_lib}
    return index_lib

In [54]:
indices  = [index_info_with_norm(file) for file in filenames]
indices

[{'test': '2017_act_with_percentiles.pkl',
  'mean_psi': 64.13894059433147,
  'psi_standard': 68.00828357055413,
  'top_five_states_by_psi': {'Maine': 'test_psi: 342.85714285714283, normalized_psi: 4.098297848873947',
   'Rhode Island': 'test_psi: 161.33333333333334, normalized_psi: 1.429155209279309',
   'New Hampshire': 'test_psi: 156.875, normalized_psi: 1.3635994696066833',
   'Delaware': 'test_psi: 140.0, normalized_psi: 1.115467931593475'}},
 {'test': '2018_act_with_percentiles.pkl',
  'mean_psi': 67.50503201183123,
  'psi_standard': 68.73165781044507,
  'top_five_states_by_psi': {'Maine': 'test_psi: 405.0, normalized_psi: 4.9103277694675365',
   'Rhode Island': 'test_psi: 205.83333333333334, normalized_psi: 2.0125849678033014',
   'Delaware': 'test_psi: 185.3846153846154, normalized_psi: 1.715069694635972',
   'New Hampshire': 'test_psi: 178.57142857142856, normalized_psi: 1.6159423488068214',
   'Pennsylvania': 'test_psi: 138.8235294117647, normalized_psi: 1.0376367989933057'}}

# Building a datarame for the readme

In [66]:
def index_df(file):
    df = pd.read_pickle(f'../data/{file}')
    df['participation_score_index'] = df['composite'].apply(lambda x : float(x))/df['participation']
    df['psi_mean'] = find_mean(df['participation_score_index'])
    df['psi_std'] = standard_deviation(df['participation_score_index'])
    tops = df.sort_values(by=['participation_score_index'], ascending = False).head()
    tops.reset_index(drop = True, inplace = True)
    return tops

In [67]:
index_dfs = [index_df(file) for file in filenames]

In [71]:
display(index_dfs[7])

,state,participation,evidence-based_reading_and_writing,math,composite,year,rounded,code,percentiles,participation_score_index,psi_mean,psi_std
0,North Dakota,0.02,640,643,1283,2018,1280,ND,86,64150.000000,12816.416479,16403.816494
1,Wisconsin,0.03,641,653,1294,2018,1290,WI,87,43133.333333,12816.416479,16403.816494
2,Iowa,0.03,634,631,1265,2018,1260,IA,83,42166.666667,12816.416479,16403.816494
3,Wyoming,0.03,633,625,1257,2018,1260,WY,83,41900.000000,12816.416479,16403.816494
4,Nebraska,0.03,629,623,1252,2018,1250,NE,82,41733.333333,12816.416479,16403.816494


In [76]:
for frame in range(len(index_dfs)):
    index_dfs[frame].to_pickle(f'../data/indices_{filenames[frame]}.pkl')